In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from joblib import load
from flask import Flask, request, jsonify

In [2]:
cred = credentials.Certificate('./firebase-sdk.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [22]:
# Function to fetch a user's data from Firestore
def fetch_user_data(user_id):
    doc_ref = db.collection('users').document(user_id)
    doc = doc_ref.get()
    if doc.exists:
        return doc.to_dict()
    else:
        print("No such document!")
        return None

# Function to preprocess user's data
def preprocess_user_data(user_data):
    # Assuming user_data is a dictionary containing the user's features
    df = pd.DataFrame([user_data])  # Convert dict to DataFrame
    
    # Apply the same preprocessing as your training data
    # Load label encoder and transform 'Country of Birth'
    le_country = load('label_encoder.joblib')
    df['encoded_country'] = le_country.transform(df['CountryofBirth'])
    
    # Ensure all preprocessing steps match training, including dropping unused columns
    processed_df = df.drop(['Name', 'CountryofBirth'], axis=1)
    
    # Return preprocessed user data ready for prediction
    return processed_df

# Load your trained Decision Tree model
DT_modelcv = load('classifier.joblib')

# Predict function for a user
def predict_tbpred(user_id):
    user_data = fetch_user_data(user_id)
    if user_data:
        processed_data = preprocess_user_data(user_data)
        prediction = DT_modelcv.predict(processed_data)
        return prediction[0]
    else:
        return "User data not found"


# Example usage
user_id = 'Id2CXsv9CuIUMTTGDLgg'
prediction = predict_tbpred(user_id)
print(f"Predicted tbpred for user {user_id}: {prediction}")

ValueError: X has 23 features, but LogisticRegression is expecting 22 features as input.

In [ ]:
#Update a Entry -> 

def updateDatainFirebase(uid, predict):
    doc_ref = db.collection(u'users').document(uid)
    
    doc_ref.update({"tb_pred":predict})

In [20]:
updateDatainFirebase(user_id,prediction)

TypeError: ('Cannot convert to a Firestore Value', 1, 'Invalid type', <class 'numpy.int64'>)